In [39]:
%use adventOfCode
import com.toldoven.aoc.notebook.AocClient
val aoc = AocClient.fromFile().interactiveDay(2024, 12)

In [40]:
val exampleInput = """
                    RRRRIICCFF
                    RRRRIICCCF
                    VVRRRCCFFF
                    VVRCCCJFFF
                    VVVVCJJCFE
                    VVIVCCJJEE
                    VVIIICJJEE
                    MIIIIIJJEE
                    MIIISIJEEE
                    MMMISSJEEE
                    """.trimIndent()

In [41]:
enum class Direction(val x: Int, val y: Int) {
    UP(0, -1), RIGHT(1, 0), DOWN(0, 1), LEFT(-1, 0);

    fun turnRight() = entries[ordinal.inc() % entries.size]
}

data class Vec2(val x: Int, val y: Int) {
    fun move(d: Direction) = Vec2(x + d.x, y + d.y)
    fun neighbours() = Direction.entries.map { dir -> move(dir) to dir }
}

class Grid(private val data: List<String>) {
    val rangeX = 0..data[0].lastIndex
    val rangeY = 0..data.lastIndex
    val points = rangeY.flatMap { y -> rangeX.map { x -> Vec2(x, y) } }

    operator fun contains(p: Vec2) = p.x in rangeX && p.y in rangeY
    operator fun get(p: Vec2) = if (p in this) data[p.y][p.x] else error("Not a valid grid position")
}

In [42]:
data class Region(val plots: Set<Vec2>) {
    val area = plots.size
    val perimeter: Set<Pair<Vec2, Direction>> = plots.flatMap { plot -> plot.neighbours().filterNot { (pos, _) -> pos in plots } }.toSet()
    val perimeterLength: Int = perimeter.size
    val price: Int = area * perimeterLength
    val sides = perimeter.filterNot { (pos, dir) -> (pos.move(dir.turnRight()) to dir) in perimeter }.toSet().size
    val discountedPrice = area * sides
}

fun getRegions(grid: Grid): List<Region> {

    fun findRegions(plots: List<Vec2>) = buildList<Region> {
        val unmappedPlots = plots.toMutableSet()
        while (unmappedPlots.isNotEmpty()) add(Region(buildSet<Vec2> {
            val neighboursToCheck = ArrayDeque<Vec2>()
            neighboursToCheck.add(unmappedPlots.first().also { unmappedPlots.remove(it) })
            while (neighboursToCheck.isNotEmpty()) {
                val nextPlot = neighboursToCheck.removeFirst()
                add(nextPlot)
                nextPlot.neighbours().filter { (pos, _) -> pos in unmappedPlots }.forEach { (pos, _) -> pos
                    unmappedPlots.remove(pos)
                    neighboursToCheck.add(pos)
                }
            }
        }))
    }

    val plotsByType = grid.points.groupBy { grid[it] }.values

    return plotsByType.flatMap { findRegions(it) }
}

In [43]:
val exampleGrid = Grid(exampleInput.lines())
val exampleRegions = getRegions(exampleGrid)
exampleRegions.forEach {
    println("${it.plots.firstOrNull()?.let {exampleGrid[it]}} | ${it.area} * ${it.perimeterLength} = ${it.price}")
}

R | 12 * 18 = 216
I | 4 * 8 = 32
I | 14 * 22 = 308
C | 14 * 28 = 392
C | 1 * 4 = 4
F | 10 * 18 = 180
V | 13 * 20 = 260
J | 11 * 20 = 220
E | 13 * 18 = 234
M | 5 * 12 = 60
S | 3 * 8 = 24


In [44]:
exampleRegions.sumOf { it.price }

1930

In [46]:
exampleRegions.forEach {
    println("${exampleGrid[it.plots.first()]} | ${it.area} * ${it.sides} = ${it.discountedPrice}")
}

R | 12 * 10 = 120
I | 4 * 4 = 16
I | 14 * 16 = 224
C | 14 * 22 = 308
C | 1 * 4 = 4
F | 10 * 12 = 120
V | 13 * 10 = 130
J | 11 * 12 = 132
E | 13 * 8 = 104
M | 5 * 6 = 30
S | 3 * 6 = 18


In [48]:
exampleRegions.sumOf { it.discountedPrice }

1206

In [49]:
val grid = Grid(aoc.input().lines())
val regions = getRegions(grid)

In [50]:
val part1 = regions.sumOf { it.price }
part1

1486324

In [51]:
aoc.submitPartOne(part1)

The answer is correct! Your answer: 1486324 You already completed this part.

In [52]:
val part2 = regions.sumOf { it.discountedPrice }
part2

898684

In [53]:
aoc.submitPartTwo(part2)

Your answer: 898684. 
 That's the right answer! You are one gold star closer to finding the Chief Historian. 
 You have completed Day 12! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .